# The Machine Learning Workflow

In [2]:
import numpy as np
import optuna
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import time

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

## Load Data

In [3]:
data = pd.read_csv('../data/RtmSimulation_kickstart.csv', index_col= 0)

#### Descriptive Statistics

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1000
Columns: 2114 entries, lai to w2500
dtypes: float64(2113), object(1)
memory usage: 16.1+ MB


In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

#### Descriptive Visualization

In [ ]:
columns_with_missing_values = data.columns[data.isnull().any()]
sns.heatmap(data[columns_with_missing_values].isnull(), cbar = False)
plt.show()

## Data Quality Summary
* The dataset contains 1000 rows.
* The column 'lai' is the target variable.
* The feature columns can be divided into three groups:
    * Numerical feature wetness
    * Categorical feature treeSpecies
    * 10 Numerical features of Sentinel-2A wavelengths
    * 2101 Numerical features of wavelengths
* 65 rows with at least one null value. Null values are only present in the columns 'Sentinel_2A_704.1', 'Sentinel_2A_740.5', 'Sentinel_2A_782.8', 'w469', 'w470', 'w471', 'w473', 'w474'

#### Define target and features

In [7]:
# Identify categorical and numerical columns
categorical_cols = [col for col in data.columns if data[col].dtype == 'object']
numerical_cols = [col for col in data.columns if col not in categorical_cols + ['id', 'lai']]

In [8]:
X = data.drop(['lai'], axis=1)
y = data['lai']

#### Train-Test Split
Feature engineering steps are applied separately to the train test and validation sets to avoid data leakage.

In [9]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
sns.kdeplot(y_train, label = 'y_train')
sns.kdeplot(y_val, label = 'y_val')
sns.kdeplot(y_test, label = 'y_test')
plt.legend()
plt.show()

## Feature Engineering

* We fill missing values with the median of each column.
* Categories are encoded with OrdinalEncoder and then passed to LightGBM as categorical features. LightGBM will handle the encoding internally.
* Numerical features are scaled with StandardScaler.

#### Fill missing values

In [11]:
X_train[numerical_cols] = X_train[numerical_cols].fillna(X_train[numerical_cols].median())
X_val[numerical_cols] = X_val[numerical_cols].fillna(X_train[numerical_cols].median())
X_test[numerical_cols] = X_test[numerical_cols].fillna(X_train[numerical_cols].median())

#### Encode categorical features

In [ ]:
ordinal_encoder = OrdinalEncoder()
X_train[categorical_cols] = ordinal_encoder.fit_transform(X_train[categorical_cols])
X_val[categorical_cols] = ordinal_encoder.transform(X_val[categorical_cols])
X_test[categorical_cols] = ordinal_encoder.transform(X_test[categorical_cols])

#### Scale numerical features

In [12]:
scl = StandardScaler()
X_train[numerical_cols] = scl.fit_transform(X_train[numerical_cols])
X_val[numerical_cols] = scl.transform(X_val[numerical_cols])
X_test[numerical_cols] = scl.transform(X_test[numerical_cols])

## Modelling

#### LightGBM

In [13]:
gbm_model = LGBMRegressor(random_state=42)

In [ ]:
gbm_model.fit(X_train, y_train, categorical_feature=set(categorical_cols))

In [15]:
y_pred_train = gbm_model.predict(X_train)
mse = mean_squared_error(y_train, y_pred_train)

print("Train MSE:", mse, "Train r2:", r2_score(y_train, y_pred_train))

y_pred_val = gbm_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred_val)

print("Validation MSE:", mse, "Validation r2:", r2_score(y_val, y_pred_val))

y_pred_test = gbm_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_test)

print("Test MSE:", mse, "Test r2:", r2_score(y_test, y_pred_test))

Train MSE: 0.0021111604630675903 Train r2: 0.9994071322907638
Validation MSE: 0.40356709369591093 Validation r2: 0.8889391624860117
Test MSE: 0.45539207185638764 Test r2: 0.8948183482390067


## Hyperparameter Optimization

In [17]:
def param_bounds(trial: optuna.Trial):

    return {
        # Sample an integer between 10 and 100
        "n_estimators": trial.suggest_categorical("n_estimators", [50, 100, 200, 300, 400, 500, 700, 1000]),
        "num_leaves": trial.suggest_int("num_leaves", 10, 100),
        "max_depth": trial.suggest_int("max_depth", 10, 100),
        # Sample a categorical value from the list provided
        "objective": trial.suggest_categorical(
            "objective", ["regression", "regression_l1", "huber"]
        ),
        "random_state": [42],
        # Sample from a uniform distribution between 0.3 and 1.0
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
        # Sample from a uniform distribution between 0 and 10
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 100),
        # Sample from a uniform distribution between 0 and 10
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 100),
    }

In [ ]:
def objective(trial: optuna.Trial):
    gbm_model = LGBMRegressor()
    params = param_bounds(trial)
    gbm_model.set_params(**params)

    gbm_model.fit(X_train, y_train, categorical_feature=set(categorical_cols))

    return gbm_model.score(X_val, y_val)

    
sampler = optuna.samplers.TPESampler(n_startup_trials=10, seed=42)
# Create a study
study = optuna.create_study(direction="maximize", sampler=sampler)
# Start the optimization run
study.optimize(objective, n_trials=50, show_progress_bar=True)

fig = optuna.visualization.plot_param_importances(study)
fig.show()

bo_search_trials = study.trials_dataframe()
best_params = study.best_params
best_score = study.best_value
print(bo_search_trials.sort_values("value").head())
print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")

#### Train with best parameters

In [ ]:
gbm_model = LGBMRegressor(random_state=42, **best_params)

In [ ]:
start = time.time()
gbm_model.fit(X_train, y_train, categorical_feature=set(categorical_cols))
end = time.time()
print(f"Training time: {end - start} seconds")

In [ ]:
start = time.time()
y_pred_train = gbm_model.predict(X_train)
end = time.time()
mse = mean_squared_error(y_train, y_pred_train)
print("Train MSE:", mse, "Train r2:", r2_score(y_train, y_pred_train), f"Prediction time: {end - start} seconds")

start = time.time()
y_pred_val = gbm_model.predict(X_val)
end = time.time()
mse = mean_squared_error(y_val, y_pred_val)
print("Validation MSE:", mse, "Validation r2:", r2_score(y_val, y_pred_val), f"Prediction time: {end - start} seconds")

start = time.time()
y_pred_test = gbm_model.predict(X_test)
end = time.time()
mse = mean_squared_error(y_test, y_pred_test)
print("Test MSE:", mse, "Test r2:", r2_score(y_test, y_pred_test), f"Prediction time: {end - start} seconds")

## Cross validation
We concatenate the train and validation sets to perform cross-validation on the whole dataset.

In [19]:
gbm_model = LGBMRegressor(random_state=42, **best_params)

In [ ]:
X_train_cross_val, y_train_cross_val = pd.concat([X_train, X_val]), pd.concat([y_train, y_val])

# Define the cross-validation strategy
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation for R² score
r2_scores = cross_val_score(gbm_model, X_train_cross_val, y_train_cross_val, cv=cv_strategy, fit_params={'categorical_feature': set(categorical_cols)})

# Calculate mean and standard deviation of R² scores
r2_mean = np.mean(r2_scores)
r2_std = np.std(r2_scores)

# Perform cross-validation for MSE
mse_scores = cross_val_score(gbm_model, X_train, y_train, scoring='neg_mean_squared_error', cv=cv_strategy, fit_params={'categorical_feature': set(categorical_cols)})

# Calculate mean and standard deviation of MSE scores
mse_mean = np.mean(-mse_scores)  # Negate the scores back to positive
mse_std = np.std(-mse_scores)    # Negate the scores back to positive

print(f"Mean cross-validation R²: {r2_mean:.3f} +/- {r2_std:.3f}")
print(f"Mean cross-validation MSE: {mse_mean:.3f} +/- {mse_std:.3f}")

### Final evaluation
We train the model on the whole dataset and evaluate it on the test set, to get the final performance metrics.

In [ ]:
gbm_model.fit(X_train_cross_val, y_train_cross_val, categorical_feature=set(categorical_cols))

In [ ]:
y_pred_test = gbm_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_test)

print("Test MSE:", mse, "Test r2:", r2_score(y_test, y_pred_test))

## Explainability
Gradient boosting models have a feature_importances_ attribute that can be used to get the relative importance of each feature.


In [ ]:
feat_df = pd.DataFrame(
    {
        "feature": X_train.columns,
        "importance": gbm_model.feature_importances_.ravel(),
    }
)

feat_df["_abs_imp"] = np.abs(feat_df.importance)
feat_df = feat_df.sort_values("_abs_imp", ascending=False).drop(
    columns="_abs_imp"
)

feat_df = feat_df.sort_values(by="importance", ascending=False).head(15)
feat_df.plot(x="feature", y="importance", kind="bar", color="blue", )